In [1]:
# Import some libraries
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
# Import Seaborn
import seaborn as sns

# This allows plots to appear on the IPython notebook.
%matplotlib inline

In [2]:
df = pd.read_csv("data/riyadh_route_edges_am.txt", sep=" ", index_col=0)
print df.shape 
df.head(3)

(21508, 8)


,source,target,volume,capacity,voc,degree,free_travel_time,travel_time
eid,,,,,,,,
101,1,2,67.2220,2850,0.023587,158,0.268594,0.268594
20415,1,5748,53.1765,2850,0.018658,108,1.238560,1.238560
14254,1,9437,262.0150,2850,0.091935,406,1.354030,1.354040


In [3]:
# Let's define an empty undirected graph.
RG = nx.Graph()

# We also define the values for the edge attributes
keys = ['volume', 'capacity', 'voc', 'degree', 'free_travel_time', 'travel_time']
i = 0
for row in df.itertuples(index=False):    
    i +=1
    # If we use all the nodes, plotting the network takes a very long time!
    if i < 10000:
        values = row[2:]
        # We create a dictionary with the keys and row values
        edge_attributes = dict(zip(keys, values))
        my_tuple = (row[0], row[1])
        # We add the edge to the graph
        RG.add_edge(*my_tuple, attr_dict=edge_attributes)
# Analysis the degrees use nx.degree() function        
degrees = nx.degree(RG)

# ***Remember to cancel this when you change i to large numbers. This is only for test print the variables.***
    ####print edge_attributes
    ##{'capacity': 2850, 'degree': 108, 'voc': 0.018658399999999999, 'volume': 53.176499999999997, 
    ##'travel_time': 1.2385600000000001, 'free_travel_time': 1.2385600000000001}
    ####print values
    #(53.176499999999997, 2850, 0.018658399999999999, 108, 1.2385600000000001, 1.2385600000000001)
    ####print degrees
    ##{1: 2, 2: 1, 5748: 1}"

In [ ]:
# Plotting might take a bit! We can stand up and stretch for a bit!

###plt.figure(figsize=(4,4))
###P = nx.spring_layout(RG)
    
#  Here we multiply the degree value times 10 to see the difference in size on our plot.
###node_sizes = [v * 5 for v in degrees.values()]
###node_colors = range(len(RG.nodes()))
###print node_colors
###nx.draw(RG, pos=P, node_size=node_sizes, width=.5, node_color=node_colors);

In [ ]:
# If I pick any two intersections, is it possible to find a route
# between them? Check if the network is connected.
print "*Is the road network connected? "+ str(nx.is_connected(RG)),

# Here we generate all the shortest paths from the graph. Records in Dictionary "paths"
paths = nx.shortest_path(RG)

# Since all the paths are stored in a big dictionary, we can access the shortest path of every node by its index or name!
# We can start by printing all the paths from node_id 1
###print "*All Paths starting from Node '1':"
###print str(paths[1]) ## \"paths\" is a dictionary with format of {node_id:}
###print "\n"


# Since within node_id 1, it connects to node_id 9950, we can count the number of nodes that exist between both
print "Count Paths (length) from Node'1' to Node'9950': " + str(len(paths[1][9950]))
print "Namely: " + str(paths[1][9950])

In [ ]:
# Import json, Read riyadh network node data to get st_x, st_y (Lng, Lat)
import json
Nodes = pd.read_csv("data/riyadh_nodes.txt", sep=" ") ## <nrows> define the total rows that are read
print "* Total Nodes Numbers read is: " + str(len(Nodes)) + "\n"
Nodes.head(3)

In [ ]:
# Now construct the multi-tier dictionary structures.
# Build the 1st tier dictionary, called \"geoDataSet\".\n",
geoDataSet  = {}
geoDataSet["type"] = "FeatureCollection"
geoDataSet["features"] = []

# Build a dictionary "Coordinate" {id:[st_x,st_y]} that join node id with st_x and st_y. 
Coordinate = {}
for i in range(len(Nodes["id"])):  # i =0,1,2,...    
    idx = Nodes["id"][i]           # idx = 1,2,3,... In fact 'idx' is 'id' in dataframe "Nodes"
    Coordinate[idx]=[Nodes["st_x"][i],Nodes["st_y"][i]]

print "Length of dict 'Coordinate': " + str(len(Coordinate)) ## Test Print
print Coordinate[50]  ## Test Print"

In [ ]:
for i in range(len(Coordinate)):         # i =0,1,2,...,    
    id_o = Nodes["id"][i]                  # idx = 1,2,3,... In fact 'idx' is 'id' of dataframe "Nodes"
    ###print "1* When i= "+str(i)+ ", Orientation Node= "+str(id_o)
    steps = len(paths[id_o].keys())  # Count how many successful lines connected to a node, including itself
    if steps >1:
        ###print "2* All Paths to Ori_node are "+str(paths[id_o])  # when idx=1, len(paths[idx])=3
        ###print len(paths[idx].keys())       # when idx=1, len(path[idx].keys())=3
        destination = [] # Define a list "destination" [] that use to record all the destinations
        #print paths[idx]
        #print Coordinate[idx]

        ## Get all the nodes id that connect with node i.
        for key in paths[id_o]: # when idx=1,paths[1]={1:[1],2:[1, 2],5748:[1, 5748]}, key=1,then 2,then 5748       
            destination.append(key)               
        ###print "3* Total destination is"+ str(destination)+"\n"

        for index in range(steps):        
            id_des = destination[index]
            ###print "4.1** When index= " + str(index)+"， Destination Node= " + str(id_des)
            tmp_dict = {}
            tmp_dict["type"] = "Feature"
            tmp_dict["geometry"] = {}
            tmp_dict["geometry"]["type"] = "LineString"
            tmp_dict["geometry"]["coordinates"] = []        
            tmp_coor=[Coordinate[id_o]] ## Already insert one origin point to avoid single point situation
            ##print "4.2** O_Node_"+str(id_des) +"'s coordination is"+ str(Coordinate[id_o])     
            for x in paths[id_o][id_des]:           
                tmp_coor.append(Coordinate[x])            
            ##print "4.3** Temporary Destination List=" + str(tmp_coor)        
            tmp_dict["geometry"]["coordinates"].append(tmp_coor)
            ##print "4.4** tmp_dict are: "+str(tmp_dict)+"\n"      
            geoDataSet["features"].append(tmp_dict)

        ##print "4*" + str(geoDataSet)+ "\n"
print "Caculation Done"

In [ ]:
for i in [1,10,100,1000,10000]:         # i =0,1,2,...,    
    id_o = Nodes["id"][i]                  # idx = 1,2,3,... In fact 'idx' is 'id' of dataframe "Nodes"
    ###print "1* When i= "+str(i)+ ", Orientation Node= "+str(id_o)
    steps = len(paths[id_o].keys())  # Count how many successful lines connected to a node, including itself
    if steps >1:
        ###print "2* All Paths to Ori_node are "+str(paths[id_o])  # when idx=1, len(paths[idx])=3
        ###print len(paths[idx].keys())       # when idx=1, len(path[idx].keys())=3
        destination = [] # Define a list "destination" [] that use to record all the destinations
        #print paths[idx]
        #print Coordinate[idx]

        ## Get all the nodes id that connect with node i.
        for key in paths[id_o]: # when idx=1,paths[1]={1:[1],2:[1, 2],5748:[1, 5748]}, key=1,then 2,then 5748       
            destination.append(key)               
        ###print "3* Total destination is"+ str(destination)+"\n"

        for index in range(steps):        
            id_des = destination[index]
            ###print "4.1** When index= " + str(index)+"， Destination Node= " + str(id_des)
            tmp_dict = {}
            tmp_dict["type"] = "Feature"
            tmp_dict["geometry"] = {}
            tmp_dict["geometry"]["type"] = "LineString"
            tmp_dict["geometry"]["coordinates"] = []        
            tmp_coor=[Coordinate[id_o]] ## Already insert one origin point to avoid single point situation
            ##print "4.2** O_Node_"+str(id_des) +"'s coordination is"+ str(Coordinate[id_o])     
            for x in paths[id_o][id_des]:           
                tmp_coor.append(Coordinate[x])            
            ##print "4.3** Temporary Destination List=" + str(tmp_coor)        
            tmp_dict["geometry"]["coordinates"].append(tmp_coor)
            ##print "4.4** tmp_dict are: "+str(tmp_dict)+"\n"      
            geoDataSet["features"].append(tmp_dict)

        ##print "4*" + str(geoDataSet)+ "\n"
print "Caculation Done"

In [ ]:
json.dumps(geoDataSet)

with open( '%snodes.json' %('10000'), 'w' ) as f:
    f.write( json.dumps(geoDataSet))

print "You are all set"